<a href="https://colab.research.google.com/github/maancham/Research-Materials/blob/main/Movielens_trimming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import pandas as pd
import numpy as np
import re
from scipy import stats

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install wget
!pip install csv2tsv
import wget

In [3]:
%%capture
wget.download('https://files.grouplens.org/datasets/movielens/ml-25m.zip')
!unzip ml-25m.zip

In [165]:
df = pd.read_csv('/content/ml-25m/ratings.csv')
movie_df = pd.read_csv('/content/ml-25m/movies.csv')

In [166]:
def extract_year(title):
  if(len(title) > 7):
    year = title[-7:]
    return year[year.find("(")+1:year.find(")")]
  else:
    return None

movie_df['year'] = movie_df['title'].apply(extract_year)
movie_df['year'] = pd.to_numeric(movie_df['year'], errors='coerce')
movie_df = movie_df[movie_df['year'].notna()]
movie_df['year'] = movie_df.loc[:, 'year'].astype(int)


movie_df['title'] = movie_df['title'].str.replace('\(.*$', '')
movie_df['genres'] = movie_df['genres'].str.replace('|', ' ')

movie_df = movie_df[movie_df['year'] >= 1970]
movieId_list = movie_df['movieId'].to_list()

<ipython-input-166-0950ff6328c9>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  movie_df['title'] = movie_df['title'].str.replace('\(.*$', '')
<ipython-input-166-0950ff6328c9>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_df['genres'] = movie_df['genres'].str.replace('|', ' ')


In [167]:
df = df[df['movieId'].isin(movieId_list)]
by_user_rating = df.groupby(by = 'userId').count().rating
low_userIds = by_user_rating[by_user_rating < 20].index.to_list()
df = df[~df['userId'].isin(low_userIds)]

### Dataset extension part:

In [168]:
new_user_df = pd.read_csv('/content/drive/MyDrive/research/new user ratings/new_user_houmch.csv', 
                          index_col=0)
new_ratings = new_user_df[['userId', 'movieId', 'rating', 'timestamp']]
new_df = pd.concat([df, new_ratings])
assert(len(df) + len(new_user_df) == len(new_df))

col_names = ['user_id:token','item_id:token','rating:float', 'timestamp:float']
new_df.columns = col_names


file_name = 'ml-25m.inter'
path = 'drive/MyDrive/research/atomic files/' + file_name
new_df.to_csv(path, sep="\t", index=False)

In [169]:
item_cols = ['movieId', 'title', 'year', 'genres']
movie_df = movie_df[item_cols]

icol_names = ['item_id:token', 'movie_title:token_seq', 'release_year:token', 'class:token_seq']
movie_df.columns = icol_names

file_name = 'ml-25m.item'
path = 'drive/MyDrive/research/atomic files/' + file_name
movie_df.to_csv(path, index=False)